# Setup Files

In [1]:
# Class Files
from imutils.video import VideoStream
import torch
import numpy as np
import time
import cv2
from PIL import Image
from facenet_pytorch import MTCNN
from fastai.vision import *
import youtube_dl
import imutils
import pathlib
import scipy.misc
import uuid
from multiprocessing import Process, Manager, Queue
import multiprocessing as mp
import dlib

#Class for a Mask Predictor
class maskPredictor_fastai:
    def __init__(self):
        #defaults.device = torch.device('cpu')
        classes = ['noMask','withMask']
        path = 'C:/Projects/Mask Detector/data'
        
        #Create blank databunch
        data2 = ImageDataBunch.single_from_classes(path, classes, ds_tfms=get_transforms(), size=224).normalize(imagenet_stats)
        
        #Create the learner
        learn = cnn_learner(data2, models.resnet50, metrics=error_rate)
        #Load it with the pretrained RESNET Weights 
        #'res34_stage2_1 best one yet
        #'res50_stage1_v3 best one yet
        learn.load('res50_stage3_v3')
        #learn.load('vgg19_stage1_v1')
        self.model = learn
        
    def maskPredict(self,face):
        img = Image(pil2tensor(face, dtype=np.float32).div_(255))
         
        tic = time.perf_counter() 
        #Make prediction
        while True:
            try:
                 
                pred_class,pred_idx,outputs = self.model.predict(img)
                score = outputs.max()
                break
                
            except ZeroDivisionError:
                print('Computation Error')
                score = 0
                pred_idx = 0
                return None
        toc = time.perf_counter()       
        
        print(f"Mask Inference:{toc - tic:0.4f}s")
                
        return score, pred_idx
        
#class for Face Detection using FastMTCNN
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
    
    def __call__(self, frames):
        #print("I, FastMTCNN, have been called")
        if self.resize != 1:
            frames = cv2.resize(frames, (int(frames.shape[1] * self.resize), int(frames.shape[0] * self.resize)))
        
        #results = self.mtcnn.detect(frames[::self.stride])
        tic= time.time() 
        boxes, results = self.mtcnn.detect(frames, landmarks=False)
        toc=time.time()
    
        #print('Face Detection Inference is:' + str(toc-tic))
        return [boxes, results]


#class for Face Detection using Standard MTCNN    q    
class stdMTCNN(object):  
    def __init__(self, resize=1, *args, **kwargs):
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
    
    def __call__(self, frames):
        if self.resize != 1:
            frames = cv2.resize(frames, (int(frames.shape[1] * self.resize), int(frames.shape[0] * self.resize)))
        
        #results = self.mtcnn.detect(frames[::self.stride])
        tic= time.time() 
        boxes, results = self.mtcnn.detect(frames, landmarks = False)
        toc=time.time()
        print('Face Detection Inference is:' + str(toc-tic))
        
        return [boxes, results]

#Load in prediction Functions

### Face
def detect_face(frame, faceNet, draw = True):
    # initialize our results list
    results = []
    
    #The higher the better face detections works
    img_size = 300
    
    if (frame is None):
        print('No frame detected')
        return None
    # grab the dimensions of the frame and then construct a blob
    (h, w) = frame.shape[:2]
    
    # pass the blob through the network and obtain the face detections
    blob = cv2.dnn.blobFromImage(frame, 1.0, (img_size, img_size),(104.0, 177.0, 123.0))
    # Pass frame through Input
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2] 
        minConf = 0.6 
        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        #compute the (x, y)-coordinates of the bounding box for

        if confidence > minConf:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            #Detect if any negative values and turn to 0
            box[box < 0] = 1
            
            (startX, startY, endX, endY) = box.astype("int")
            #Store results of face coordinates
            d = [startX, startY, endX, endY]
            
            
        
            #Draw rectangle around face
            if draw is True:
                cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            #In cases that there are more than one face per frame, it appends to a list
            d = [startX, startY, endX, endY]
            
            if any(t < 0 for t in d):
                for i,p in enumerate(d):
                    print (i)
                    if d[i] < 0:
                        d[i] = 0
            results.append(d)       
    return results

def detect_face_MTCNN(frame, faceNet, face_tracker, draw = True):
    results = []
  
    if (frame is None):
        print('No frame detected')
        return None
    
    (h, w) = frame.shape[:2]
    pad_f = 0.03
    pad_x = int(w * pad_f)
    pad_y = int(h * pad_f)
    
    frame_p = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    minConf = 0.5
    

    #Check if any objects are tracked; If none, conduct a detect Face Algorithm
    if len(face_tracker) == 0:
        #Pass frame through network and come out with bounding box in (face)
        #We do the predictions here
        [face, score] = faceNet(frame_p)
        if (face is None or face.size == 0):
            print('No face detected')
            cv2.imshow('frame',frame)
            key = cv2.waitKey(1) & 0xFF
            return None
    
        #Change negative values to 0 (Bug in in the model)
        face[face < 0] = 0
    
        #Resize facebounding box to original size
        if faceNet.resize != 1:
            face = np.multiply(face,(1/faceNet.resize))
        
        #For each detected face
        for i in range(0, face.shape[0]):

            #Check if results is more thatn confidence
            if score[i] > minConf:
                # Unpack
                (startX, startY, endX, endY) = face[i].astype(int)
                

                # Start a tracking object for each face detected
                t = dlib.correlation_tracker()
                rect = dlib.rectangle(startX-pad_x, startY-pad_y, endX+pad_x, endY+pad_y)
                t.start_track(frame, rect)

                #update the tracker list
                face_tracker.append(t)

                #Draw rectangle around face
                if draw:
                    cv2.rectangle(frame, (startX-pad_x, startY-pad_y), (endX+pad_x, endY+pad_y),(0, 0, 255), 2)

                d = [startX-pad_x, startY-pad_y, endX+pad_x, endY+pad_y]
                #d = [startX, startY, endX, endY]
                results.append(d)


    elif len(face_tracker) != 0:
    #If trackers are not empty
        #print("Tracking Faces")
        for t in face_tracker:
            t.update(frame)
            pos = t.get_position()

            startX = int(pos.left())
            startY = int(pos.top())
            endX = int(pos.right())
            endY = int(pos.bottom())

            if draw:
                cv2.rectangle(frame, (startX-pad_x, startY-pad_y), (endX+pad_x, endY+pad_y),(0, 0, 255), 2)

            d = [startX-pad_x, startY-pad_y, endX+pad_x, endY+pad_y]
            #d = [startX, startY, endX, endY]
            results.append(d)    
        
    return results
    

### MASK
def detect_mask(face_box, frame, maskNet, save, minConf = 0.8, draw = True):
    pred_dict = {0: 'No Mask',1: 'With Mask'}
    results = []
    p = 0
    
   #Check if face has data
    if [x for x in (face_box, frame) if x is None]:
        print('No face detected 1')
        return None
    
    #Iterate over the returned results
    for i in face_box:
        (startX, startY, endX, endY) = face_box[p]
        p = p+1
        #Crop out just the face from the frame
        face = frame[startY:endY, startX:endX]
        if [x for x in (face, frame) if x is None]:
            print('No face detected 2')
            return None
        
        #Pass through Mask Detector Model and Time
        mask_results = maskNet.maskPredict(face = face)
        
        if mask_results is None:
            return None

        #Get results from tensors and convert to a list
        d = [mask_results[1].item(), mask_results[0].item()]

        #Only annotate ones which have a good confidence
        if d[1] > minConf:
        
        #Script for saving image in folder
            if save:
                print('called')
                size = (224, 224)
                face = cv2.resize(face, size)
                face = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
                im = Image.fromarray(face)

                if d[0] == 1:#If Face has Mask
                    outfile = '%s/%s.jpg' % ('train_data/withMask', 'withMask' + str(uuid.uuid4()))
                    im.thumbnail(size)
                    im.save(outfile)
                    print('saved image')
                elif d[0] == 0: #If face has noMask
                    outfile = '%s/%s.jpg' % ('train_data/noMask', 'noMask' + str(uuid.uuid4()))
                    im.thumbnail(size)
                    im.save(outfile)
                    print('saved image')

            #Annotate if draw is True
            if draw:

                #Change color of text according to results
                if d[0] == 1:
                    text_color = (0,255,0)
                elif d[0] == 0:
                    text_color = (0,0,255)

                text = "{}: {:.2f}%".format(pred_dict[d[0]], d[1] * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.putText(frame, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, text_color, 2)

        results.append(d)

    return results

# Playing A Webcam Stream

In [3]:
import cv2
import numpy as np
import pathlib
import scipy.misc
import uuid
import os
import imutils


class WebCamPlayer:    
    def __init__(self, faceNet='ResCaffe'):
        print("Initializing")
        #Load Predict setup files
        current_dir = pathlib.Path().absolute()
        
        #Choosing the face Detection System
        if faceNet == 'ResCaffe':
            #Load up hass Face Detector    
            prototxtPath =  os.path.join(current_dir,'face_detector','deploy.prototxt')
            weightsPath = os.path.join(current_dir,'face_detector','res10_300x300_ssd_iter_140000.caffemodel')
            faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
            self.faceNetType = 'ResCaffe' 
        
        elif faceNet == 'SFD':
            #Load up hass SFD Detector   
            prototxtPath =  os.path.join(current_dir,'face_detector','deploy_sfd.prototxt')
            weightsPath = os.path.join(current_dir,'face_detector', 'SFD.caffemodel')
            faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
            self.faceNetType = 'SFD' 
        
        elif faceNet == 'fastMTCNN':
            faceNet = FastMTCNN(
                stride=4,
                resize=0.5,
                margin=14,
                factor=0.6,
                keep_all=True,
                select_largest=False,
                device='cuda:0'
            )
            self.faceNetType = 'fastMTCNN'
            self.resize = faceNet.resize
            
        elif faceNet == 'stdMTCNN':
            faceNet = stdMTCNN(
                select_largest=False,
                resize=0.5,
                margin=20,
                factor=0.6,
                keep_all=True,
                device='cuda:0'
            )
            self.faceNetType = 'stdMTCNN'
            self.resize = faceNet.resize
            
        print(str(self.faceNetType)+' initialized as the Detection')
        #Choosing the Mask Detection Version
        maskNet = maskPredictor_fastai()
        
        self.faceNet = faceNet
        self.maskNet = maskNet
        
    
    def webcam_play(self, detect = True, draw = True, save = False, skip_frames = 0):
        vs = VideoStream(src=1).start()
        print("Starting Web Cam Feed")
        time.sleep(2.0)
        frame_count = 0
        M = 0
        while True:
            frame_count = frame_count + 1
            # grab the frame from the video stream
            # to have a maximum width of 400 pixels
            frame = vs.read()
            
            if (frame is None):
                print('No frame detected')
                break
            #Resize The frame
            frame = imutils.resize(frame, width=600)

            #Script to only apply detection after N frames if skip_Frames is not set to 0
            if not skip_frames == 0:
                M = frame_count % skip_frames

            if detect and M==0:
                if self.faceNetType in ['ResCaffe','SFD']:
                    face_box = detect_face(frame, self.faceNet)
                    if face_box is None:
                        continue
                    results = detect_mask(face_box, frame, self.maskNet,save)

                elif self.faceNetType in ['fastMTCNN','stdMTCNN']:
                    face_box = detect_face_MTCNN(frame, self.faceNet, draw)
                    if face_box is None:
                        continue
                    results = detect_mask(face_box, frame, self.maskNet,save)

            # display frame
            cv2.namedWindow("frame")
            cv2.imshow('frame', frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                break
                    
        # release VideoCapture
        vs.stop()
        vs.stream.release()

In [6]:
vid = WebCamPlayer(faceNet = 'ResCaffe')
vid.webcam_play()

Initializing
ResCaffe initialized as the Detection
Starting Web Cam Feed


In [4]:
vs.stop()
vs.stream.release()

NameError: name 'vs' is not defined

# Playing Video

In [2]:

class vidPlayer:
    def __init__(self, video_url, faceNet='ResCaffe'):
        print("Initializing")
        #Load Predict setup files
        current_dir = pathlib.Path().absolute()
        
        #Choosing the face Detection System
        if faceNet == 'ResCaffe':
            #Load up hass Face Detector    
            prototxtPath =  os.path.join(current_dir,'face_detector','deploy.prototxt')
            weightsPath = os.path.join(current_dir,'face_detector','res10_300x300_ssd_iter_140000.caffemodel')
            faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
            self.faceNetType = 'ResCaffe' 
        
        elif faceNet == 'SFD':
            #Load up hass SFD Detector   
            prototxtPath =  os.path.join(current_dir,'face_detector','deploy_sfd.prototxt')
            weightsPath = os.path.join(current_dir,'face_detector', 'SFD.caffemodel')
            faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
            self.faceNetType = 'SFD' 
        
        elif faceNet == 'fastMTCNN':
            faceNet = FastMTCNN(
                stride=4,
                resize=0.5,
                margin=14,
                factor=0.6,
                keep_all=True,
                select_largest=False,
                device='cpu'
            )
            self.faceNetType = 'fastMTCNN'
            self.resize = faceNet.resize
            
        elif faceNet == 'stdMTCNN':
            faceNet = stdMTCNN(
                select_largest=False,
                resize=0.25,
                margin=20,
                factor=0.6,
                keep_all=True,
                post_process = False,
                device='cpu'
            )
            self.faceNetType = 'stdMTCNN'
            self.resize = faceNet.resize
            
        print(str(self.faceNetType)+' initialized as the Detection')
        #Choosing the Mask Detection Version
        maskNet = maskPredictor_fastai()
        
        self.faceNet = faceNet
        self.maskNet = maskNet
        
        #Video URL
        self.video_url = video_url
        
    
    def youtube_play(self, detect = True, draw = True, save = False, skip_frames = 0, save_vid = False):
        #ydl_opts = {'no_check_certificate': True}      #Additional options for video stream
        ydl_opts = {}
        ydl = youtube_dl.YoutubeDL(ydl_opts)                           # create youtube-dl object
        info_dict = ydl.extract_info(self.video_url, download=False)         # set video url, extract video information

        # get video formats available
        formats = info_dict.get('formats',None)

        for f in formats:
            
            # Set Resolution
            #if f.get('format',None) == '720p':
            if f.get('format_note',None) == '720p':

                #get the video url
                url = f.get('url',None)
                
                # open url with opencv
                cap = cv2.VideoCapture(url)
                # check if url was opened
                if not cap.isOpened():
                    print('Error: Video not opened')
                    continue

                #Initialize object tracking arrays
                #Start frame count
                frame_count = 0
                face_tracker = []
                while True:
                    # modulo variable
                    M = 0
                    
                    # read frame
                    ret, frame = cap.read()
                    
                    #Start Frame Counting
                    frame_count = frame_count + 1
                    
                    # check if frame is empty
                    if not ret:
                        print('No frame found')
                        break
                    
                    #Script that clears up the trackers every N frames so the faceNet can happen
                    if not skip_frames == 0:
                        if (frame_count%skip_frames) == 0:
                        #Clear up  face tracker
                            face_tracker = []
                        
                    if detect:
                        #Pass frames into which Detector Type
                        if self.faceNetType in ['ResCaffe','SFD']:
                            
                            face_box = detect_face(frame, self.faceNet)
                            if face_box is None:
                                continue
                            results = detect_mask(face_box, frame, self.maskNet,save)
                       
                        elif self.faceNetType in ['fastMTCNN','stdMTCNN']:
                    
                            #Detect face #Bit of a heavy operation
                            face_box = detect_face_MTCNN(frame, self.faceNet, face_tracker, draw)
                            
                            #If no faces are detected go back to start, but before that, update the latest frame
                            if face_box is None:
                                print("No face Detected 4")
                                continue
                            results = detect_mask(face_box, frame, self.maskNet, save)
                            
                    if save_vid:
                        cv2.imwrite("misc/vidimages/"+str(frame_count)+".jpg", frame)  # save frame as JPG file
                        print()
                        print('saving frame')
                        
                    # display frame
                    
                    cv2.imshow('frame', frame)
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord("q"):
                        break

                # release VideoCapture
                cap.release()
        cv2.destroyAllWindows()


In [3]:
vid =vidPlayer('https://www.youtube.com/watch?v=YbSqC1r0nUc',faceNet = 'stdMTCNN')
#https://www.youtube.com/watch?v=AYrTp08p5Cs American Protest
#https://www.youtube.com/watch?v=bVPAAvfpw9U Philippine Market 
#https://www.youtube.com/watch?v=YbSqC1r0nUc Philippine News Report Video
#https://www.youtube.com/watch?v=g-dyHadto2Y&t=38s
#vid =vidPlayer('https://www.youtube.coqqm/watch?v=Um0Z7Lo_jHs',faceNet = 'MTCNN')
#https://www.youtube.com/watch?v=_xA57XCjAtI high res?
#https://www.youtube.com/watch?v=ShlI5_sR2do
#https://www.youtube.com/watch?v=_wox36bFDqE
#Add in argument save = True If you want to save images classified into folders of withMask or NoMask
#https://www.youtube.com/watch?v=7Yiq_g0bGjk Good Diverse Video of the Philippines
#vid.youtube_play(detect=1, draw = False, save = True, skip_frames = 30)
vid.youtube_play(detect=1, draw = True, save = False, skip_frames = 15, save_vid=False)

Initializing
stdMTCNN initialized as the Detection
[youtube] YbSqC1r0nUc: Downloading webpage
Face Detection Inference is:0.06249403953552246
No face detected
No face Detected 4
Face Detection Inference is:0.04686903953552246
Mask Inference:1.8855s
Mask Inference:0.0426s
Mask Inference:0.0422s
Mask Inference:0.0330s
Mask Inference:0.0390s
Mask Inference:0.0409s
Mask Inference:0.0345s
Mask Inference:0.0339s
Mask Inference:0.0387s
Mask Inference:0.0362s
Mask Inference:0.0396s
Mask Inference:0.0422s
Mask Inference:0.0371s
Face Detection Inference is:0.05097007751464844
No face detected
No face Detected 4
Face Detection Inference is:0.060964345932006836
No face detected
No face Detected 4
Face Detection Inference is:0.04197549819946289
No face detected
No face Detected 4
Face Detection Inference is:0.04997134208679199
No face detected
No face Detected 4
Face Detection Inference is:0.045972347259521484
No face detected
No face Detected 4
Face Detection Inference is:0.043973445892333984
No f

In [16]:
cv2.destroyAllWindows()